In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('dataset.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35815 entries, 0 to 35814
Data columns (total 45 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              35815 non-null  object 
 1   spkid           35815 non-null  int64  
 2   full_name       35815 non-null  object 
 3   pdes            35815 non-null  int64  
 4   name            17872 non-null  object 
 5   prefix          0 non-null      float64
 6   neo             35815 non-null  object 
 7   pha             35815 non-null  object 
 8   H               35815 non-null  float64
 9   diameter        26432 non-null  float64
 10  albedo          26414 non-null  float64
 11  diameter_sigma  26329 non-null  float64
 12  orbit_id        35815 non-null  object 
 13  epoch           35814 non-null  float64
 14  epoch_mjd       35814 non-null  float64
 15  epoch_cal       35814 non-null  float64
 16  equinox         35814 non-null  object 
 17  e               35814 non-null 

In [10]:
df.isnull().sum()


neo                  0
pha                  0
H                    0
diameter          9383
albedo            9401
diameter_sigma    9486
epoch                1
epoch_mjd            1
epoch_cal            1
e                    1
a                    1
q                    1
i                    1
om                   1
w                    1
ma                   1
ad                   1
n                    1
tp                   1
tp_cal               1
per                  1
per_y                1
moid                 1
moid_ld              1
sigma_e              1
sigma_a              1
sigma_q              1
sigma_i              1
sigma_om             1
sigma_w              1
sigma_ma             1
sigma_ad             1
sigma_n              1
sigma_tp             1
sigma_per            1
class                1
rms                  1
dtype: int64

In [8]:
df = df.drop(['name', 'prefix'], axis=1)
df = df.drop(['id', 'spkid','full_name', 'equinox','orbit_id','pdes'], axis=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35815 entries, 0 to 35814
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   neo             35815 non-null  object 
 1   pha             35815 non-null  object 
 2   H               35815 non-null  float64
 3   diameter        26432 non-null  float64
 4   albedo          26414 non-null  float64
 5   diameter_sigma  26329 non-null  float64
 6   epoch           35814 non-null  float64
 7   epoch_mjd       35814 non-null  float64
 8   epoch_cal       35814 non-null  float64
 9   e               35814 non-null  float64
 10  a               35814 non-null  float64
 11  q               35814 non-null  float64
 12  i               35814 non-null  float64
 13  om              35814 non-null  float64
 14  w               35814 non-null  float64
 15  ma              35814 non-null  float64
 16  ad              35814 non-null  float64
 17  n               35814 non-null 

In [11]:
df = df.dropna()


In [12]:
df.isna().sum()


neo               0
pha               0
H                 0
diameter          0
albedo            0
diameter_sigma    0
epoch             0
epoch_mjd         0
epoch_cal         0
e                 0
a                 0
q                 0
i                 0
om                0
w                 0
ma                0
ad                0
n                 0
tp                0
tp_cal            0
per               0
per_y             0
moid              0
moid_ld           0
sigma_e           0
sigma_a           0
sigma_q           0
sigma_i           0
sigma_om          0
sigma_w           0
sigma_ma          0
sigma_ad          0
sigma_n           0
sigma_tp          0
sigma_per         0
class             0
rms               0
dtype: int64

In [13]:
from sklearn import preprocessing

In [14]:
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['pha'])
df['pha'] = label_encoder.transform(df['pha']) 

In [15]:
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['neo'])
df['neo'] = label_encoder.transform(df['neo']) 
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['class'])
df['class'] = label_encoder.transform(df['class']) 

In [17]:
from sklearn.model_selection import train_test_split
y = df['pha']
X= df.drop(['pha'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [23]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0, k_neighbors=2)

X= df.drop(['pha'], axis=1)
y = df['pha']

column = X.columns

print("До балансировки")
print(X.shape)
print(y.value_counts())

os_data_X, os_data_y = os.fit_resample(X, y)
os_data_X = pd.DataFrame(data=os_data_X, columns=column)
os_data_y = pd.DataFrame(data=os_data_y, columns=['pha'])

print("После балансировки")
print(os_data_X.shape)
print(os_data_y.value_counts())

До балансировки
(26328, 36)
0    26310
1       18
Name: pha, dtype: int64
После балансировки
(52620, 36)
pha
0      26310
1      26310
dtype: int64


In [29]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(os_data_X, os_data_y)
model_pred=model.predict(x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [30]:
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, precision_score
print("accuracy:",accuracy_score(y_test,model_pred))
print("recall:",recall_score(y_test,model_pred))






accuracy: 0.9997468033928345
recall: 1.0


In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(model_pred, y_test)

array([[7889,    0],
       [   2,    8]])